In [304]:
from bs4 import BeautifulSoup
import nltk, re, pymorphy2
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")
morph = pymorphy2.MorphAnalyzer()

In [305]:
pages = []

i = 1
while True:
    try:
        with open(f"p{i}.html", "r", encoding="utf-8") as f:
            pages.append(f.read())
        i += 1
    except FileNotFoundError:
        break

In [306]:
def extract_data(page, reviews=[]):

    page_soup = BeautifulSoup(page, "html.parser")
    
    for container in page_soup.find_all("div", {"class": "ow-opinion ow-opinions__item", "data-role": "opinion"}) + page_soup.find_all("div", {"class": "ow-opinion ow-opinion_popular ow-opinions__item", "data-role": "opinion"}):
        
        stars = len(container.find_all("span", {"class": "star-rating__star", "data-state": "selected"}))
        
        text_chunks = container.find_all("div", {"class": "ow-opinion__text-desc"})
        
        texts = []
        for i in range(len(text_chunks)):
            
            stroki = []
            for stroka in text_chunks[i].find_all("p"):
                stroki.append(stroka.text)
            
            text_chunks[i] = "\n".join(stroki)
            if not text_chunks[i] in ("{{Достоинства}}", "{{Недостатки}}", "{{Комментарий}}"):
                texts.append(text_chunks[i])
                
        texts = "\n".join(texts)
        
        if len(texts) > 10:
            reviews.append({"text": texts, "rating": stars})
    return reviews

In [307]:
reviews = []

for page in pages:
    reviews = extract_data(page, reviews)
    
print(len(reviews))
#print(reviews)

215


In [308]:
def tokenize_review(review):
    tokens = nltk.word_tokenize(review["text"])
    review["tokens"] = []
    
    for token in tokens:
        if re.search("[А-Яа-яЁё]", token) is not None:
            review["tokens"].append(token.strip("-"))
    
    return review

def lemmatize_review(review):
    for i in range(len(review["tokens"])):
        review["tokens"][i] = morph.parse(review["tokens"][i])[0].normal_form
    
    filtered_tokens = []
    for i in range(len(review["tokens"])):
        if review["tokens"][i] not in russian_stopwords:
            filtered_tokens.append(review["tokens"][i])
    
    review["tokens"] = filtered_tokens
    
    return review

def preedit(review):
    review = tokenize_review(review)
    review = lemmatize_review(review)
    return review

In [309]:
for i in range(len(reviews)):
    reviews[i] = preedit(reviews[i])
    
print(reviews)

[{'text': 'Цена = качество\nПока не поняла, если они за такую цену\nВыбирала в этом ценовом сегменте, остановилась на этих из-за положительных отзывов. Купила не для частого прослушивания, например, пока в автобусе едешь или голосовое смс прослушать, вот. Для этого наушники в самый раз. Заодно купила оригинальный чехол, удобно, мне нравится.', 'rating': 10, 'tokens': ['цена', 'качество', 'пока', 'понять', 'цена', 'выбирать', 'это', 'ценовый', 'сегмент', 'остановиться', 'из-за', 'положительный', 'отзыв', 'купить', 'частый', 'прослушивание', 'например', 'пока', 'автобус', 'ехать', 'голосовой', 'смс', 'прослушать', 'это', 'наушник', 'самый', 'заодно', 'купить', 'оригинальный', 'чехол', 'удобно', 'нравиться']}, {'text': 'Коробка была в пленке, однако попытка зарядить наушники завершилась неудачей, кейс не реагирует никак. Пойду сдавать по гарантии.', 'rating': 2, 'tokens': ['коробка', 'плёнка', 'однако', 'попытка', 'зарядить', 'наушник', 'завершиться', 'неудача', 'кейс', 'реагировать', 'ни

In [313]:
def make_freq_dict(reviews, min):
    all_tokens = {}
    for review in reviews:
        for token in review["tokens"]:
            if token in all_tokens:
                all_tokens[token].append(review["rating"])
            else:
                all_tokens[token] = [review["rating"]]
    
    popular_tokens = {}
    for token in all_tokens:
        if len(all_tokens[token]) >= min:
            all_tokens[token].sort()
            popular_tokens[token] = all_tokens[token]
    
    return popular_tokens

popular_tokens = make_freq_dict(reviews, min=4)
popular_tokens

{'цена': [2,
  2,
  2,
  4,
  4,
  4,
  4,
  4,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10],
 'качество': [2,
  2,
  2,
  2,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10],
 'пока': [2,
  2,
  4,
  4,
  4,
  6,
  6,
  8,
  8,
  8,
  8,
  8,
  8,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10],
 'понять': [4, 6, 6, 8, 10, 10],
 'выбирать': [8, 8, 10, 10, 10, 10, 10, 10],
 'это': [2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  

In [315]:
def assign_polarity(tokens):
    
    def count_mean(a):
        return round(sum(a)/len(a), 2)
    
    def count_median(a):
        i = int(len(a)//2)
        return a[i] if len(a) % 2 == 1 else count_mean([a[i], a[i-1]])
    
    def count_dispersion(a, mean):
        deviations = []
        for rating in a:
            deviations.append((rating-mean)**2)
        return count_mean(deviations)
    
    markers = {"positive": set(), "negative": set()}
    
    for token in tokens:
        mean = count_mean(tokens[token])
        median = count_median(tokens[token])
        dispersion = count_dispersion(tokens[token], mean)
        
        if dispersion < 3:
            if median >= 9 and len(tokens[token]) > 5:
                markers["positive"].add(token)
            elif median <= 7:
                markers["negative"].add(token)
    
    return markers

markers = assign_polarity(popular_tokens)
markers

{'positive': {'вполне',
  'выбирать',
  'диапазон',
  'длительный',
  'довольный',
  'каждый',
  'настроить',
  'немного',
  'особенно',
  'отличный',
  'противник',
  'сборка',
  'супер',
  'удобно',
  'удобство',
  'чистый',
  'шея'},
 'negative': {'амбошюр',
  'брак',
  'вылезти',
  'звукоизоляция',
  'крайне',
  'момент',
  'начаться',
  'неудобный',
  'писк',
  'подушка',
  'прийти',
  'сдать',
  'технология',
  'тугой'}}

In [316]:
test_pages = []

for x in "ABCDE":
    with open(f"p{x}.html", "r", encoding="utf-8") as f:
        test_pages.append(f.read())
        
test_reviews = []
for test_page in test_pages:
    test_reviews = extract_data(test_page, test_reviews)

for i in range(len(test_reviews)):
    test_reviews[i] = preedit(test_reviews[i])
    
print(test_reviews)

[{'text': '+ Звук хороший.\n+ Аккуратный дизайн.\n+ Шумодав работает (интересная функция).\n+ Приятная девшука шепчет (Power On, Off и т.д.) при нажатии на кнопку вкл.\n+ Не упадут с ушей, гарантировано.\n- Эти наушники однозначно нацелены на использование для смартфонов, аудио-плееров, авиасамолетов.\n- В комплекте за такую стоимость НЕТ Bluetooth Устройства для PC.\n- Невозможно выключить шумодав вручную без приложения на Android Sony Head....\n- Амбушюры не внушают долгосрочного доверия.\n- Материал обертки ткани и сам ободок, дешевее некуда.\n- Шнур для ПК: это прям максимальная экономия на проводе. 40-60 см т.е. меньше метра. Вам надо сидеть рядом с системником в подмышку его.\n- Шнур для зарядки 7-10 см В КОМПЛЕКТЕ ДАЖЕ ЗАРЯДНИК НЕ ПОЛОЖИЛИ! ЭКОНОМИЯ!\n- Давят зачетно.\n- Не стоят они столько.\nВЫ ТОЛЬКО ПОДУМАЙТЕ!!!! НЕТ УСТРОЙСТВА BLUETOOTH В КОМПЛЕКТЕ!!! ЕГО НЕТ! ПОКУПАТЬ НУЖНО ОТДЕЛЬНО.', 'rating': 6, 'tokens': ['звук', 'хороший', 'аккуратный', 'дизайн', 'шумодать', 'работать

In [317]:
def predict_mood(test_review):
    counter_pos = 0
    counter_neg = 0
    for token in test_review["tokens"]:
        if token in markers["positive"]:
            counter_pos += 1
        elif token in markers["negative"]:
            counter_neg -= 1
    print(test_review['rating'], counter_pos, counter_neg, sep="\t")

for test_review in test_reviews:
    predict_mood(test_review)

6	0	0
4	1	0
10	2	0
6	0	0
10	1	-1
10	0	0
10	0	0
10	0	0
10	1	0
8	0	0
10	1	0
6	1	-2
10	2	0
10	0	0
8	0	-1
10	1	0
10	2	0
10	2	0
6	2	0
10	0	0
10	1	0
10	1	0
10	2	0
6	0	0
4	0	0
10	0	0
4	0	0
8	1	0
6	0	0
2	1	0
8	2	0
8	0	0
6	1	0
8	0	0
8	0	0
10	0	0
8	1	0
8	0	-1
10	0	0
8	2	0
8	0	0
2	1	0
8	0	0
10	3	0
10	0	0
10	0	0
4	0	-1
10	0	0
8	1	0
2	0	0
6	0	0
4	3	-1
8	1	-3
8	1	0
6	1	-1
